In [35]:
#préparation du dataset

import os, glob
os.getcwd()
import pandas as pd
data_tweets = pd.read_csv('french_tweets.csv')
print(data_tweets.shape)
data_tweets.head()



(1526724, 2)


,label,text
0,0,"- Awww, c'est un bummer. Tu devrais avoir davi..."
1,0,Est contrarié qu'il ne puisse pas mettre à jou...
2,0,J'ai plongé plusieurs fois pour la balle. A ré...
3,0,Tout mon corps a des démangeaisons et comme si...
4,0,"Non, il ne se comporte pas du tout. je suis en..."


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import time


In [3]:
print("longueur de verteur : ")
for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),lowercase = False)
    print("ngram_range",(min_N, max_N),":")
    print(V.fit_transform(data_tweets['text']).getnnz())

longueur de verteur : 
ngram_range (1, 1) :
19373177
ngram_range (1, 2) :
38226814
ngram_range (1, 3) :
55631117


In [4]:
print("longueur de verteur sans stopwords: ")
for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),lowercase = False,stop_words=["french"])
    print("ngram_range",(min_N, max_N),":")
    print(V.fit_transform(data_tweets['text']).getnnz())

longueur de verteur sans stopwords: 
ngram_range (1, 1) :
19373092
ngram_range (1, 2) :
38226643
ngram_range (1, 3) :
55630860


In [5]:
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import MultinomialNB 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

liste_classifieurs= [
    ["Perceptron", Perceptron(eta0=0.1, random_state=0)],
    ["MultinomialNB", MultinomialNB()],
    ["Logistic Regression", LogisticRegression()],
    ["linear_svc", LinearSVC()],
    ["Random Forest",RandomForestClassifier(n_estimators=250,max_depth=4, random_state=0)]
    
]
import warnings 
warnings.filterwarnings("ignore")

In [46]:
Resultat=[]

In [37]:
import json
import os
chemin_expes = "tweets_original.json"

if os.path.exists(chemin_expes):
    f = open(chemin_expes)
    dic_expes = json.load(f)
    f.close()
else:
    dic_expes = {}
print("Expériences stockées : %s"%len(dic_expes))

Expériences stockées : 15


In [39]:
en_minuscules,enlever_stopwords  = False, False
from sklearn.metrics import classification_report 

for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),lowercase=False)
    X = V.fit_transform(data_tweets['text'])
    y = data_tweets['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print( "-" * 15)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for nom, algo in liste_classifieurs:
        expe = str([nom, min_N, max_N, enlever_stopwords, en_minuscules])
        if expe in dic_expes:
            print("  Déjà vu")
            score = dic_expes[expe]
            print(expe, "\n",score[0],"\n",score[1],"\n",score[2])
        else:
            T1=time.time()
            clf = algo.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            dic_expes[expe] = [score]
            print('  %s classifier : %.4f'%(nom, score))
            pred = clf.predict(X_test)
            nom_classes=["Possitive","Négative"]
            report = classification_report(y_test,pred,target_names=nom_classes)
            T2=time.time()
            calcul_time = ("Temps_execution(secondes)",T2 - T1)
            dic_expes[expe].append(report)
            dic_expes[expe].append(calcul_time)
            print(report)
            print(calcul_time)
            

---------------
Ngram_range : (1, 1)
  Déjà vu
['Perceptron', 1, 1, False, False] 
 0.7151618495342977 
               precision    recall  f1-score   support

   Possitive       0.71      0.74      0.73    231214
    Négative       0.72      0.69      0.70    226804

    accuracy                           0.72    458018
   macro avg       0.72      0.71      0.71    458018
weighted avg       0.72      0.72      0.71    458018
 
 ['Temps_execution(secondes)', 2.087865114212036]
  Déjà vu
['MultinomialNB', 1, 1, False, False] 
 0.7656642315367518 
               precision    recall  f1-score   support

   Possitive       0.75      0.80      0.78    231214
    Négative       0.78      0.73      0.75    226804

    accuracy                           0.77    458018
   macro avg       0.77      0.77      0.77    458018
weighted avg       0.77      0.77      0.77    458018
 
 ['Temps_execution(secondes)', 0.6462728977203369]
  Déjà vu
['Logistic Regression', 1, 1, False, False] 
 0.791032230

In [50]:
import json
json_str = json.dumps(dic_expes, indent=4) # 缩进4字符
with open(chemin_expes, 'a') as json_file:
	json_file.write(json_str)

In [51]:
liste_resultats =[[score[0], nom_expe] for nom_expe, score in dic_expes.items()]
for res in sorted(liste_resultats,reverse=True):
    print(res)
    Resultat.append(res)

[0.8097804016436035, "['linear_svc', 1, 3, True, False]"]
[0.8097804016436035, "['linear_svc', 1, 3, False, False]"]
[0.8091472387548088, "['Logistic Regression', 1, 2, False, False]"]
[0.8090031396146004, "['Logistic Regression', 1, 2, True, False]"]
[0.8086014086782616, "['Logistic Regression', 1, 3, True, False]"]
[0.8071953504010759, "['Logistic Regression', 1, 3, False, False]"]
[0.8032238907641185, "['linear_svc', 1, 2, True, False]"]
[0.8032238907641185, "['linear_svc', 1, 2, False, False]"]
[0.79103223017436, "['Logistic Regression', 1, 1, False, False]"]
[0.7910191302525228, "['Logistic Regression', 1, 1, True, False]"]
[0.7881742638935587, "['MultinomialNB', 1, 3, True, False]"]
[0.7881698972529464, "['MultinomialNB', 1, 3, False, False]"]
[0.7864450742110572, "['MultinomialNB', 1, 2, True, False]"]
[0.786442890890751, "['MultinomialNB', 1, 2, False, False]"]
[0.78393425585894, "['linear_svc', 1, 1, True, False]"]
[0.7839298892183276, "['linear_svc', 1, 1, False, False]"]
[0.

In [54]:
chemin_expes = "tweets_sw.json"

if os.path.exists(chemin_expes):
    f = open(chemin_expes)
    dic_sans = json.load(f)
    f.close()
else:
    dic_sans = {}
print("Expériences stockées : %s"%len(dic_sans))

Expériences stockées : 30


In [56]:
from sklearn.metrics import classification_report 
en_minuscules,enlever_stopwords  = False, True

for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),lowercase=False,stop_words=["french"])
    X = V.fit_transform(data_tweets['text'])
    y = data_tweets['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print( "-" * 15)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for nom, algo in liste_classifieurs:
        expe = str([nom, min_N, max_N, enlever_stopwords, en_minuscules])
        if expe in dic_expes:
            print("  Déjà vu")
            score = dic_expes[expe]

            print(expe, "\n",score[0],"\n",score[1],"\n",score[2])
        else:
            T1=time.time()
            clf = algo.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            dic_expes[expe] = [score]
            print('  %s classifier : %.4f'%(nom, score))
            pred = clf.predict(X_test)
            nom_classes=["Possitive","Négative"]
            report = classification_report(y_test,pred,target_names=nom_classes)
            T2=time.time()
            calcul_time = ("Temps_execution(secondes)",T2 - T1)
            dic_expes[expe].append(report)
            dic_expes[expe].append(calcul_time)
            print(report)
            print(calcul_time)


---------------
Ngram_range : (1, 1)
  Déjà vu
['Perceptron', 1, 1, True, False] 
 0.7110419241165195 
               precision    recall  f1-score   support

   Possitive       0.71      0.72      0.72    231214
    Négative       0.71      0.70      0.71    226804

    accuracy                           0.71    458018
   macro avg       0.71      0.71      0.71    458018
weighted avg       0.71      0.71      0.71    458018
 
 ('Temps_execution(secondes)', 1.9634089469909668)
  Déjà vu
['MultinomialNB', 1, 1, True, False] 
 0.7656707814976704 
               precision    recall  f1-score   support

   Possitive       0.75      0.80      0.78    231214
    Négative       0.78      0.73      0.75    226804

    accuracy                           0.77    458018
   macro avg       0.77      0.77      0.77    458018
weighted avg       0.77      0.77      0.77    458018
 
 ('Temps_execution(secondes)', 0.6620018482208252)
  Déjà vu
['Logistic Regression', 1, 1, True, False] 
 0.79101913025

In [23]:
json_str = json.dumps(dic_sans, ensure_ascii=False, indent=4) # 缩进4字符
with open(chemin_expes, 'a') as json_file:
	json_file.write(json_str)


In [24]:
liste_resultats =[[score[0], nom_expe] for nom_expe, score in dic_expes.items()]
for res in sorted(liste_resultats,reverse=True):
    print(res)
    Resultat.append(res)

[0.8097804016436035, "['linear_svc', 1, 3, True, False]"]
[0.8097804016436035, "['linear_svc', 1, 3, False, False]"]
[0.8091472387548088, "['Logistic Regression', 1, 2, False, False]"]
[0.8090031396146004, "['Logistic Regression', 1, 2, True, False]"]
[0.8086014086782616, "['Logistic Regression', 1, 3, True, False]"]
[0.8071953504010759, "['Logistic Regression', 1, 3, False, False]"]
[0.8032238907641185, "['linear_svc', 1, 2, True, False]"]
[0.8032238907641185, "['linear_svc', 1, 2, False, False]"]
[0.79103223017436, "['Logistic Regression', 1, 1, False, False]"]
[0.7910191302525228, "['Logistic Regression', 1, 1, True, False]"]
[0.7881742638935587, "['MultinomialNB', 1, 3, True, False]"]
[0.7881698972529464, "['MultinomialNB', 1, 3, False, False]"]
[0.7864450742110572, "['MultinomialNB', 1, 2, True, False]"]
[0.786442890890751, "['MultinomialNB', 1, 2, False, False]"]
[0.78393425585894, "['linear_svc', 1, 1, True, False]"]
[0.7839298892183276, "['linear_svc', 1, 1, False, False]"]
[0.

In [57]:
chemin_expes = "tweets_lower.json"

if os.path.exists(chemin_expes):
    f = open(chemin_expes)
    dic_expes = json.load(f)
    f.close()
else:
    dic_expes = {}
print("Expériences stockées : %s"%len(dic_expes))

Expériences stockées : 0


In [58]:
en_minuscules,enlever_stopwords  = True, False

for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),lowercase=True)
    X = V.fit_transform(data_tweets['text'])
    y = data_tweets['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print( "-" * 15)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for nom, algo in liste_classifieurs:
        expe = str([nom, min_N, max_N, enlever_stopwords, en_minuscules])
        if expe in dic_expes:
            print("  Déjà vu")
            print(expe, "\n",score[0],"\n",score[1],"\n",score[2])
            score = dic_expes[expe]
        else:
            T1=time.time()
            clf = algo.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            dic_expes[expe] = [score]
            print('  %s classifier : %.4f'%(nom, score))
            pred = clf.predict(X_test)
            nom_classes=["Possitive","Négative"]
            report = classification_report(y_test,pred,target_names=nom_classes)
            T2=time.time()
            calcul_time = ("Temps_execution(secondes)",T2 - T1)
            dic_expes[expe].append(report)
            dic_expes[expe].append(calcul_time)
            print(report)
            print(calcul_time)
            

---------------
Ngram_range : (1, 1)
  Perceptron classifier : 0.7138
              precision    recall  f1-score   support

   Possitive       0.72      0.71      0.71    231214
    Négative       0.71      0.72      0.71    226804

    accuracy                           0.71    458018
   macro avg       0.71      0.71      0.71    458018
weighted avg       0.71      0.71      0.71    458018

('Temps_execution(secondes)', 1.9993071556091309)
  MultinomialNB classifier : 0.7633
              precision    recall  f1-score   support

   Possitive       0.75      0.80      0.77    231214
    Négative       0.78      0.73      0.75    226804

    accuracy                           0.76    458018
   macro avg       0.76      0.76      0.76    458018
weighted avg       0.76      0.76      0.76    458018

('Temps_execution(secondes)', 0.6397099494934082)
  Logistic Regression classifier : 0.7896
              precision    recall  f1-score   support

   Possitive       0.80      0.78      0.79

In [62]:
json_str = json.dumps(dic_expes, ensure_ascii=False, indent=4) # 缩进4字符
with open(chemin_expes, 'a') as json_file:
	json_file.write(json_str)

In [60]:
liste_resultats =[[score[0], nom_expe] for nom_expe, score in dic_expes.items()]
for res in sorted(liste_resultats,reverse=True):
    print(res)
    Resultat.append(res)

[0.8106275299224048, "['Logistic Regression', 1, 3, False, True]"]
[0.8102105157439228, "['linear_svc', 1, 3, False, True]"]
[0.8097804016436035, "['linear_svc', 1, 3, True, False]"]
[0.8097804016436035, "['linear_svc', 1, 3, False, False]"]
[0.8091472387548088, "['Logistic Regression', 1, 2, False, False]"]
[0.8090031396146004, "['Logistic Regression', 1, 2, True, False]"]
[0.8086166919204049, "['Logistic Regression', 1, 2, False, True]"]
[0.8086014086782616, "['Logistic Regression', 1, 3, True, False]"]
[0.8071953504010759, "['Logistic Regression', 1, 3, False, False]"]
[0.8032238907641185, "['linear_svc', 1, 2, True, False]"]
[0.8032238907641185, "['linear_svc', 1, 2, False, False]"]
[0.8027239104140012, "['linear_svc', 1, 2, False, True]"]
[0.79103223017436, "['Logistic Regression', 1, 1, False, False]"]
[0.7910191302525228, "['Logistic Regression', 1, 1, True, False]"]
[0.7896349051783991, "['Logistic Regression', 1, 1, False, True]"]
[0.7884449956115261, "['MultinomialNB', 1, 3, 

In [63]:
chemin_expes = "tweets_lower_sw.json"

if os.path.exists(chemin_expes):
    f = open(chemin_expes)
    dic_sans = json.load(f)
    f.close()
else:
    dic_sans = {}
print("Expériences stockées : %s"%len(dic_sans))

Expériences stockées : 60


In [30]:
en_minuscules,enlever_stopwords  = True, True

for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),lowercase=True,stop_words=["french"])
    X = V.fit_transform(data_tweets['text'])
    y = data_tweets['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print( "-" * 15)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for nom, algo in liste_classifieurs:
        expe = str([nom, min_N, max_N, enlever_stopwords, en_minuscules])
        if expe in dic_expes:
            print("  Déjà vu")
            print(expe, "\n",score[0],"\n",score[1],"\n",score[2])
            score = dic_expes[expe]
        else:
            T1=time.time()
            clf = algo.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            dic_expes[expe] = [score]
            print('  %s classifier : %.4f'%(nom, score))
            pred = clf.predict(X_test)
            nom_classes=["Possitive","Négative"]
            report = classification_report(y_test,pred,target_names=nom_classes)
            T2=time.time()
            calcul_time = ("Temps_execution(secondes)",T2 - T1)
            dic_expes[expe].append(report)
            dic_expes[expe].append(calcul_time)
            print(report)
            print(calcul_time)
            

---------------
Ngram_range : (1, 1)
  Perceptron classifier : 0.7083
              precision    recall  f1-score   support

   Possitive       0.70      0.73      0.72    231214
    Négative       0.71      0.69      0.70    226804

    accuracy                           0.71    458018
   macro avg       0.71      0.71      0.71    458018
weighted avg       0.71      0.71      0.71    458018

('Temps_execution(secondes)', 1.9757401943206787)
  MultinomialNB classifier : 0.7633
              precision    recall  f1-score   support

   Possitive       0.75      0.80      0.77    231214
    Négative       0.78      0.73      0.75    226804

    accuracy                           0.76    458018
   macro avg       0.76      0.76      0.76    458018
weighted avg       0.76      0.76      0.76    458018

('Temps_execution(secondes)', 0.6524312496185303)
  Logistic Regression classifier : 0.7892
              precision    recall  f1-score   support

   Possitive       0.80      0.78      0.79

In [31]:
json_str = json.dumps(dic_expes, indent=4) 
with open(chemin_expes, 'a') as json_file:
	json_file.write(json_str)

In [33]:
liste_resultats =[[score[0], nom_expe] for nom_expe, score in dic_expes.items()]
for res in sorted(liste_resultats,reverse=True):
    print(res)
    Resultat.append(res)

[0.8106275299224048, "['Logistic Regression', 1, 3, False, True]"]
[0.8102105157439228, "['linear_svc', 1, 3, False, True]"]
[0.8101908658611671, "['linear_svc', 1, 3, True, True]"]
[0.8097804016436035, "['linear_svc', 1, 3, True, False]"]
[0.8097804016436035, "['linear_svc', 1, 3, False, False]"]
[0.8092171050046068, "['Logistic Regression', 1, 3, True, True]"]
[0.8091472387548088, "['Logistic Regression', 1, 2, False, False]"]
[0.8090031396146004, "['Logistic Regression', 1, 2, True, False]"]
[0.8086166919204049, "['Logistic Regression', 1, 2, False, True]"]
[0.8086014086782616, "['Logistic Regression', 1, 3, True, False]"]
[0.8084704094598902, "['Logistic Regression', 1, 2, True, True]"]
[0.8071953504010759, "['Logistic Regression', 1, 3, False, False]"]
[0.8032238907641185, "['linear_svc', 1, 2, True, False]"]
[0.8032238907641185, "['linear_svc', 1, 2, False, False]"]
[0.8027239104140012, "['linear_svc', 1, 2, False, True]"]
[0.8027042605312456, "['linear_svc', 1, 2, True, True]"]


In [34]:
for res in sorted(Resultat,reverse=True):
    print(res)

[0.8106275299224048, "['Logistic Regression', 1, 3, False, True]"]
[0.8106275299224048, "['Logistic Regression', 1, 3, False, True]"]
[0.8102105157439228, "['linear_svc', 1, 3, False, True]"]
[0.8102105157439228, "['linear_svc', 1, 3, False, True]"]
[0.8101908658611671, "['linear_svc', 1, 3, True, True]"]
[0.8097804016436035, "['linear_svc', 1, 3, True, False]"]
[0.8097804016436035, "['linear_svc', 1, 3, True, False]"]
[0.8097804016436035, "['linear_svc', 1, 3, True, False]"]
[0.8097804016436035, "['linear_svc', 1, 3, False, False]"]
[0.8097804016436035, "['linear_svc', 1, 3, False, False]"]
[0.8097804016436035, "['linear_svc', 1, 3, False, False]"]
[0.8097804016436035, "['linear_svc', 1, 3, False, False]"]
[0.8092171050046068, "['Logistic Regression', 1, 3, True, True]"]
[0.8091472387548088, "['Logistic Regression', 1, 2, False, False]"]
[0.8091472387548088, "['Logistic Regression', 1, 2, False, False]"]
[0.8091472387548088, "['Logistic Regression', 1, 2, False, False]"]
[0.8091472387